In [1]:
from stellargraph import datasets, utils
from tensorflow.keras import callbacks, optimizers, losses, metrics, regularizers, Model
import numpy as np
import pandas as pd

from stellargraph.mapper import KGTripleGenerator
from stellargraph.layer import DistMult

from IPython.display import HTML

c:\Users\Shantam Saxena\AppData\Local\Programs\Python\Python38\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
fb15k = datasets.FB15k()

fb15k_graph, fb15k_train, fb15k_test, fb15k_valid = fb15k.load()

In [3]:
fb15k_train[fb15k_train['source'] == '/m/0frmb1' ]

,source,label,target
15864,/m/0frmb1,/people/person/employment_history./business/em...,/m/0gsg7
28108,/m/0frmb1,/people/person/employment_history./business/em...,/m/09d5h
33681,/m/0frmb1,/people/person/education./education/education/...,/m/0bkj86
101635,/m/0frmb1,/people/person/employment_history./business/em...,/m/05l71
132806,/m/0frmb1,/people/person/education./education/education/...,/m/027ybp
138100,/m/0frmb1,/tv/tv_personality/tv_regular_appearances./tv/...,/m/05ll37
158848,/m/0frmb1,/people/person/education./education/education/...,/m/02h4rq6
193209,/m/0frmb1,/american_football/football_player/former_team...,/m/0fbq2n
213048,/m/0frmb1,/people/person/employment_history./business/em...,/m/01q0kg
235205,/m/0frmb1,/american_football/football_player/former_team...,/m/05tg3


In [ ]:
fb15k_graph.create_graph_schema()

In [7]:

epochs = 100
embedding_dimension = 100
negative_samples = 2

In [8]:
fb15k_gen = KGTripleGenerator(
    fb15k_graph, batch_size=len(fb15k_train) // 10  # ~100 batches per epoch
)

fb15k_distmult = DistMult(
    fb15k_gen,
    embedding_dimension=embedding_dimension,
    embeddings_regularizer=regularizers.l2(1e-8),
)

fb15k_inp, fb15k_out = fb15k_distmult.in_out_tensors()

fb15k_model = Model(inputs=fb15k_inp, outputs=fb15k_out)
fb15k_model.compile(
    optimizer=optimizers.Adam(lr=0.001),
    loss=losses.BinaryCrossentropy(from_logits=True),
    metrics=[metrics.BinaryAccuracy(threshold=0.0)],
)

c:\Users\Shantam Saxena\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
fb15k_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 100)       1495100     ['input_1[0][0]',                
                                                                  'input_3[0][0]']            

In [10]:

fb15k_train_gen = fb15k_gen.flow(
    fb15k_train, negative_samples=negative_samples, shuffle=True
)
fb15k_valid_gen = fb15k_gen.flow(fb15k_valid, negative_samples=negative_samples)

In [11]:
fb15k_es = callbacks.EarlyStopping(monitor="val_loss", patience=50)
fb15k_history = fb15k_model.fit(
    fb15k_train_gen,
    validation_data=fb15k_valid_gen,
    epochs=epochs,
    callbacks=[fb15k_es],
    verbose=0,
)